In [1]:
import numpy as np
import pandas as pd
import random
from scipy.stats import norm
from collections import deque, namedtuple

import plotly.express as px
import plotly.graph_objs as go

import keras
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense
from tqdm.auto import tqdm

In [2]:
###https://github.com/johntfoster/bspline
!pip install bspline
import bspline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.2/84.2 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for bspline: filename=bspline-0.1.1-py3-none-any.whl size=84483 sha256=29b719f5c3f891e7d9feaab184926880f81059b31a825f1bdf1bac0aa398f048
  Stored in directory: /root/.cache/pip/wheels/3c/ab/0a/70927853a6d9166bc777922736063a6f99c43a327c802f9326
Successfully built bspline


In [3]:
def terminal_payoff(st, k, exercise_type='c'):
    '''
    Calculate terminal payoff for a call/put option
    Args:
        st: final stock price
        k: strike
        exercise_type: 'c' for call and 'p' for put
    Examples:
        st_eg = np.linspace(1, 100, 100)
        payoff_eg = terminal_payoff(st_eg, 50)
        fig = px.scatter(x=st_eg, y=payoff_eg, title='Terminal Payoff w/ Stock Price for Call Option')
        fig.show()
    '''
    return np.maximum(st - k, 0) if exercise_type == 'c' else np.maximum(k - st, 0)

In [4]:
def delta_calc(s, k, maturity, r, sigma, exercise_type='c'):
    '''
    Calculate delta of a call/put option based on BSM formula, specifically N(d1)
    Args:
        ercise_type: 'c' for call and 'p' for put
    Examples:
        s_eg = np.linspace(1, 100, 100)
        d_eg = delta_calc(s_eg, k=50, maturity=1.0, r=0.03, sigma=0.18)

        fig = px.scatter(x=s_eg, y=d_eg, title='Delta w/ Stock Price for Call Option')
        fig.show()
    '''
    d1 = (np.log(s / k) + (r + 0.5 * sigma ** 2) * maturity) / (sigma * np.sqrt(maturity))
    if exercise_type == 'c':
        delta_calc = norm.cdf(d1, loc=0, scale=1)
    else:
        delta_calc = -norm.cdf(-d1, loc=0, scale=1)
    return delta_calc

In [5]:
def black_scholes(s, k, maturity, r, sigma, exercise_type='c'):
    '''
    Calculate price of a call/put option based on BSM formula
    Args:
        ercise_type: 'c' for call and 'p' for put
    Examples:
        s_eg = np.linspace(1, 100, 100)
        p_eg = black_scholes(s_eg, k=50, maturity=1.0, r=0.03, sigma=0.18)

        fig = px.scatter(x=s_eg, y=p_eg, title='Price w/ Stock Price for Call Option')
        fig.show()
    '''
    d1 = (np.log(s / k) + (r + 0.5 * sigma ** 2) * maturity) / (sigma * np.sqrt(maturity))
    d2 = d1 - sigma * np.sqrt(maturity)

    if exercise_type == 'c':
        price = s * norm.cdf(d1, loc=0, scale=1) - k * np.exp(-r * maturity) * norm.cdf(d2, loc=0, scale=1)
    else:
        price = k * np.exp(-r * maturity) * norm.cdf(-d2, loc=0, scale=1) - s * norm.cdf(-d1, loc=0, scale=1)
    return price

In [6]:
def show_sel_path(input_df, sel_path, value_name='Value'):
    df = input_df.reset_index().rename(columns={'index':'path_id'})
    df = df.melt(id_vars=['path_id'], value_vars=input_df.columns, var_name='Time Steps', value_name=value_name)

    fig = px.line(df[df['path_id'].isin(sel_path)], x='Time Steps', y=value_name, color='path_id')
    return fig

In [7]:
np.random.seed(100)
num_of_path = 2**14-1 # number of paths

s0 = 100  # initial stock price
mu = 0.05  # drift
sigma = 0.15  # constant volatility
r = 0.03  # interest rate
expiry = 0.25  # expiry, 3-month
num_of_timestep = 63  # number of time steps expiry*1/252
delta_t = expiry / num_of_timestep  # time interval, daily 1/252
strike = s0  # assume at-the-money option
exercise_type = 'c' # 'c' for call option and 'p' for put option

gamma = np.exp(- r * delta_t)  # discount factor
risk_aversion = 0.1 # risk aversion  \kappa
risk_lambda = risk_aversion / 2.0 # 0.001  #\lambda

data_cutoff = int(num_of_path * 0.7) #split data into training and test

In [8]:
# standard normal random variable Z
rand_number = pd.DataFrame(np.random.randn(num_of_path, num_of_timestep), index=range(1, num_of_path + 1),
                           columns=range(1, num_of_timestep + 1))

# stock price, rows are MC paths, columns are time steps
spot_df = pd.DataFrame([], index=range(1, num_of_path + 1), columns=range(num_of_timestep + 1))
spot_df[0] = s0
for t in range(1, num_of_timestep + 1):
    spot_df[t] = spot_df[t - 1] * np.exp((mu - 0.5 * sigma**2) * delta_t + sigma * np.sqrt(delta_t) * rand_number[t])

In [9]:
delta_df = pd.DataFrame([], index=range(1, num_of_path + 1), columns=range(num_of_timestep + 1))
for t in range(num_of_timestep + 1):
    delta_df.loc[:,t] = delta_calc(spot_df.loc[:,t], strike, expiry - delta_t * t, r, sigma, exercise_type)

<ipython-input-9-c360fc2313dc>:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  delta_df.loc[:,t] = delta_calc(spot_df.loc[:,t], strike, expiry - delta_t * t, r, sigma, exercise_type)


In [10]:
option_value_df = pd.DataFrame([], index=range(1, num_of_path + 1), columns=range(num_of_timestep + 1))
for t in range(num_of_timestep):
    option_value_df.loc[:,t] = black_scholes(spot_df.loc[:,t], strike, expiry - delta_t * t, r, sigma, exercise_type)
option_value_df.iloc[:,-1] = terminal_payoff(spot_df.iloc[:,-1], strike, exercise_type)

<ipython-input-10-4ac0524e3e4a>:3: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  option_value_df.loc[:,t] = black_scholes(spot_df.loc[:,t], strike, expiry - delta_t * t, r, sigma, exercise_type)
<ipython-input-10-4ac0524e3e4a>:4: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  option_value_df.iloc[:,-1] = terminal_payoff(spot_df.iloc[:,-1], strike, exercise_type)


In [11]:
sel_path = np.random.choice(num_of_path, 5)
display(show_sel_path(spot_df, sel_path, 'Spot'))
display(show_sel_path(delta_df, sel_path, 'Delta'))
display(show_sel_path(option_value_df, sel_path, 'Option Value'))

In [12]:
delta_s = np.exp(-r * delta_t) * spot_df.loc[:,1:num_of_timestep].values - spot_df.loc[:,0:num_of_timestep-1]
delta_c = np.exp(-r * delta_t) * option_value_df.loc[:,1:num_of_timestep].values - option_value_df.loc[:,0:num_of_timestep-1]

spot_df_train, spot_df_test = spot_df[:data_cutoff], spot_df[data_cutoff:]
delta_df_train, delta_df_test = delta_df[:data_cutoff], delta_df[data_cutoff:]
delta_s_train, delta_s_test = delta_s[:data_cutoff], delta_s[data_cutoff:]
delta_c_train, delta_c_test = delta_c[:data_cutoff], delta_c[data_cutoff:]

In [13]:
# delta_s_tilde = delta_s - delta_s.mean(axis=0)
# delta_c_tilde = delta_c - delta_c.mean(axis=0)

delta_s_tilde_train = delta_s_train - delta_s_train.mean(axis=0)
delta_s_tilde_test = delta_s_test - delta_s_test.mean(axis=0)

delta_c_tilde_train = delta_c_train - delta_c_train.mean(axis=0)
delta_c_tilde_test = delta_c_test - delta_c_test.mean(axis=0)

In [14]:
delta_c_tilde_test.shape

(4915, 63)

In [15]:
class Option:
  def __init__(self, s0=100, mu=0.05, sigma=0.15, r=0.03,
               expiry=0.25, num_of_timestep=63, exercise_type='c'):
    self.s0 = s0  # initial stock price
    self.mu = mu  # drift
    self.sigma = sigma  # constant volatility
    self.r = r  # interest rate
    self.expiry = expiry  # expiry, 3-month
    self.num_of_timestep = num_of_timestep  # number of time steps expiry*1/252
    self.delta_t = expiry / num_of_timestep  # time interval, daily 1/252
    self.strike = s0  # assume at-the-money option
    self.exercise_type = exercise_type  # 'c' for call option and 'p' for put option

  def compute_value(self, stock_price):
    return black_scholes(stock_price, self.strike, self.maturity, self.r, self.sigma, self.exercise_type)

In [16]:
class HedgingProblem:
  def __init__(self, ds, dc, greekd, stock_prices, num_of_timestep,
               dstilde, dctilde, option=None):
    self.option = option or Option()
    self.A = np.arange(-1.1, 0.1, 0.1)   ###discretize action from 0.1 to -1.1 with 0.1 interval
    self.ds = ds
    self.dc = dc
    self.dstilde = dstilde
    self.dctilde = dctilde
    self.greekd = greekd
    self.stock_prices = stock_prices
    self.num_simulations = len(self.ds)
    self.num_of_timestep = num_of_timestep

  def T(self, simulation, current_date, maturity):
    if current_date >= self.num_of_timestep:
      return (None, None)   # Option expires, simulation ends
    next_stock_price = self.stock_prices.iloc[simulation, current_date + 1]
    next_maturity = maturity - self.option.delta_t
    return (next_stock_price, next_maturity)

  def R(self, simulation, current_date, action):
    if current_date >= self.num_of_timestep:
      return 0
    dst = self.ds.iloc[simulation, current_date]  # ds of this simulation
    dct = self.dc.iloc[simulation, current_date]  # dc of this simulation
    dw = (action * dst) + dct  # dw of this simulation

    reward = dw - (risk_aversion / 2) * ((action * self.dstilde.iloc[simulation, current_date] +
                                          self.dctilde.iloc[simulation, current_date]) ** 2)
    return reward

In [17]:
memory_parts = ["spot", "maturity", "action", "next_spot", "next_maturity", "reward"]
Memory = namedtuple("Memory", memory_parts) # a single entry of the memory replay

class ReplayMemory:
    def __init__(self, max_length=None):
        self.max_length = max_length
        self.memory = deque(maxlen=max_length)

    def store(self, data):
        self.memory.append(data)

    def _sample(self, k):
        return random.sample(self.memory, k)

    def structured_sample(self, k):
        batch = self._sample(k)
        result = {}
        for i, part in enumerate(memory_parts):
            result[part] = np.array([row[i] for row in batch])

        return result

    def __len__(self):
        return len(self.memory)

In [18]:
hedge = HedgingProblem(ds=delta_s_train, dc=delta_c_train, greekd=delta_df_train, stock_prices=spot_df_train, num_of_timestep=num_of_timestep,
               dstilde=delta_s_tilde_train, dctilde=delta_c_tilde_train)  # set up the problem
eps = 0.4
eps_decay = 0.95
min_eps = 0.1
lr = 0.01
#gamma = np.exp(-r * hedge.option.delta_t)

In [19]:
# Set up the NN model
Qmodel = Sequential()

# Add input layer with three nodes (input_dim=3)
Qmodel.add(Dense(units=8, input_dim=3, activation='relu'))

# Add two hidden layer with 16 nodes
#for layer in range (2):
Qmodel.add(Dense(units=16, activation='relu')) ##leaky_relu
#Qmodel.add(Dense(units=16, activation='relu'))

Qmodel.add(Dense(units=8, activation='tanh'))
# Add output layer with one node
Qmodel.add(Dense(units=1, activation='linear'))

# Compile the model
Qmodel.compile(optimizer=keras.optimizers.Adam(learning_rate=0.01),
               loss='mean_squared_error')

# Display the model summary
Qmodel.summary()
#Qmodel = tf.keras.models.load_model('qmodel_20231206.keras')

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 8)                 32        
                                                                 
 dense_1 (Dense)             (None, 16)                144       
                                                                 
 dense_2 (Dense)             (None, 8)                 136       
                                                                 
 dense_3 (Dense)             (None, 1)                 9         
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [20]:
def greedy(spot, maturity, Qmodel, hedge):
  if abs(maturity) < 0.001:
    return (None, 0)
  greedy_action = None
  best_Q = float('-inf')
  for action in hedge.A:
    input_data = tf.constant([[spot, maturity, action]])
    Qvalue = Qmodel.predict(input_data, verbose=0)[0, 0]
    # print(action)
    # print(Qvalue)
    if Qvalue > best_Q:
      greedy_action = action
      best_Q = Qvalue

  return (greedy_action, best_Q)

In [21]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx]

In [22]:
replay_memory = ReplayMemory(max_length=10000)
batch_size = 50  # number of samples to draw for experience replay
min_length = 500  # minimum length of replay memory before training starts

In [33]:
n_simulations = 10

for simulation in tqdm(range(10, 15)):
  checkpoint_path = f"cp{simulation}.ckpt"
  cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                   verbose=1, save_weights_only=False, save_freq=5)
  spot = hedge.option.s0
  maturity = hedge.option.expiry

  for day in range (hedge.num_of_timestep):
    if len(replay_memory) >= min_length:
      if random.random() < eps:
        action = random.choice(hedge.A)
      else:
        action = greedy(spot, maturity, Qmodel, hedge)[0]
        print(action)
    else:
        delta_value = -1 * hedge.greekd.iloc[simulation, day]
        action = find_nearest(hedge.A, delta_value)

    rwrd = hedge.R(simulation, day, action)
    (next_spot, next_maturity) = hedge.T(simulation, day, maturity)

    memory = Memory(spot, maturity, action, next_spot, next_maturity, rwrd)
    replay_memory.store(memory)

    if len(replay_memory) >= min_length:
      batch = replay_memory.structured_sample(batch_size) # get samples from the replay memory
      input_data = []
      targets = []
      for replay_sample in range (batch_size):
        input_data.append([batch['spot'][replay_sample],
                           batch['maturity'][replay_sample],
                           batch['action'][replay_sample]])
        Q_star = batch['reward'][replay_sample] + gamma * greedy(batch['next_spot'][replay_sample],
                                                         batch['next_maturity'][replay_sample],
                                                         Qmodel, hedge)[1]
        targets.append([Q_star])

      input_data = tf.constant(input_data)
      targets = tf.constant(targets)
      Qmodel.fit(input_data, targets, epochs=1, batch_size=batch_size, verbose=1, callbacks=[cp_callback])

    (spot, maturity) = (next_spot, next_maturity)

  eps = max(min_eps, eps * eps_decay)

  0%|          | 0/5 [00:00<?, ?it/s]

-1.1
1/1 [==============================] - 0s 6ms/step - loss: 0.0548
-1.1
1/1 [==============================] - 0s 6ms/step - loss: 0.0807
-1.1
1/1 [==============================] - 0s 6ms/step - loss: 0.0546
-1.1
1/1 [==============================] - 0s 6ms/step - loss: 0.0634
-1.1

Epoch 1: saving model to cp10.ckpt
1/1 [==============================] - 1s 755ms/step - loss: 0.1287
-1.1
1/1 [==============================] - 0s 6ms/step - loss: 0.0161
-1.1
1/1 [==============================] - 0s 6ms/step - loss: 0.0909
-1.1
1/1 [==============================] - 0s 6ms/step - loss: 0.0364

Epoch 1: saving model to cp10.ckpt
1/1 [==============================] - 0s 6ms/step - loss: 0.1134


KeyboardInterrupt: ignored

In [ ]:
Qmodel.save('qmodel_20231206.keras')

In [24]:
###load model and check the performance in test data
loaded_model = tf.keras.models.load_model('qmodel_20231206.keras')
#loaded_model = tf.keras.models.load_model(f'cp{n_simulations-1}.ckpt')

In [25]:
def calc_reward_(mode = 'ql', n_evals=10):
  hedge_eval = HedgingProblem(ds=delta_s_test, dc=delta_c_test, greekd=delta_df_test, stock_prices=spot_df_test, num_of_timestep=num_of_timestep,
                dstilde=delta_s_tilde_test, dctilde=delta_c_tilde_test)  # set up the problem

  reward_calc = pd.DataFrame([], index=delta_s_test.index, columns=range(num_of_timestep))
  action_calc = pd.DataFrame([], index=delta_s_test.index, columns=range(num_of_timestep))
  for eval in tqdm(range(n_evals)):
    spot = hedge_eval.option.s0
    maturity = hedge_eval.option.expiry

    for day in range (hedge.num_of_timestep):
      if mode == 'ql':
        action = greedy(spot, maturity, loaded_model, hedge_eval)[0]
      elif mode == 'delta':
        delta_value = -1 * delta_df_test.iloc[eval, day]
        action = find_nearest(hedge_eval.A, delta_value)
      else:
        raise Exception('the mode is not supported')

      action_calc.iloc[eval, day] = action
      rwrd = hedge_eval.R(eval, day, action)
      reward_calc.iloc[eval, day] = rwrd

      (next_spot, next_maturity) = hedge_eval.T(eval, day, maturity)
      (spot, maturity) = (next_spot, next_maturity)

  return reward_calc, action_calc

In [26]:
def calc_total_reward(reward_calc):
  reward_calc = reward_calc.copy()
  total_reward = 0.0
  for t in range(num_of_timestep):
      total_reward = total_reward + np.exp(-r * delta_t * t) * reward_calc.loc[:,t].mean()
  return total_reward

In [27]:
reward_q, action_q = calc_reward_(mode = 'ql', n_evals=10)

  0%|          | 0/10 [00:00<?, ?it/s]

In [28]:
reward_delta, action_delta = calc_reward_(mode = 'delta', n_evals=10)

  0%|          | 0/10 [00:00<?, ?it/s]

In [29]:
total_reward_q = calc_total_reward(reward_q)
total_reward_delta = calc_total_reward(reward_delta)

total_reward_q, total_reward_delta

(-0.7836212936073974, -0.053355171795848276)

In [33]:
data=[
    go.Scatter(x=action_delta.columns, y=action_delta.mean(axis=0).values, line=dict(dash='dash'),  name='Delta hedge'),
    go.Scatter(x=action_q.columns, y=action_q.mean(axis=0).values, name='Q-Learning hedge'),
]
layout=go.Layout(
        title=go.layout.Title(text='Average hedge each timestep', x=0.5),
        width=800,
        height=400
    )
fig = go.Figure(data=data, layout=layout)
fig.show()

In [35]:
data=[
    go.Scatter(x=reward_delta.columns, y=reward_delta.mean(axis=0).values, line=dict(dash='dash'),  name='Delta hedge'),
    go.Scatter(x=reward_q.columns, y=reward_q.mean(axis=0).values, name='Q-Learning hedge'),
]
layout=go.Layout(
        title=go.layout.Title(text='', x=0.5), #Average reward each timestep
        width=800,
        height=400
    )
fig = go.Figure(data=data, layout=layout)
fig.update_layout(xaxis_title="Timestep", yaxis_title="Reward", margin=dict(t=10))
fig.show()